In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Суммарный вес в формате pd.DataFrame ~4GB

In [ ]:
df_train = pd.read_csv('../input/train_V2.csv')
df_test = pd.read_csv('../input/test_V2.csv')
df_sub = pd.read_csv('../input/sample_submission_V2.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
id_features = ['Id', 'groupId', 'matchId']
categorical_features = ['matchType', ]
target_feature = 'winPlacePerc'

In [ ]:
x = df_train.drop(id_features + categorical_features + [target_feature], axis=1)
y = df_train[target_feature]

In [ ]:
y[y.isnull()]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=.33)

In [ ]:
x_train.fillna(0, inplace=True)
y_train.fillna(0, inplace=True)

x_valid.fillna(0, inplace=True)
y_valid.fillna(0, inplace=True)

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(x_train, y_train)

In [ ]:
pred_train = pd.DataFrame(model.predict(x_train), x_train.index)
pred_valid = pd.DataFrame(model.predict(x_valid), x_valid.index)

In [ ]:
from sklearn.metrics import mean_absolute_error

train_error = mean_absolute_error(y_train, pred_train)
valid_error = mean_absolute_error(y_valid, pred_valid)
print(f'Train MAE: {train_error} \nValid MAE: {valid_error}')

Очевидно, что значения target ограничены [0, 1], поэтому clip улучшает score

In [ ]:
train_error = mean_absolute_error(y_train, pred_train.clip(lower=0, upper=1))
valid_error = mean_absolute_error(y_valid, pred_valid.clip(lower=0, upper=1))
print(f'Train MAE: {train_error} \nValid MAE: {valid_error}')

In [ ]:
model.intercept_

In [ ]:
coefs = [(col, coef) for col, coef in zip(x_train.columns, model.coef_)]
importance = sorted(importance, key=lambda tpl: tpl[1], reverse=True)
df_importance = pd.DataFrame(importance, columns=['feature', 'importance'])

Значения коэффициентов. Нужно помнить, что данные не нормировались, поэтому коэффициенты нельзя**** рассматривать как importance

In [ ]:
df_importance.plot(kind='bar', x='feature', y='importance')

Абсолютные значения коэффициентов

In [ ]:
df_importance['abs_importance'] = np.abs(df_importance['importance'])
df_importance.sort_values('abs_importance', ascending=False).plot(kind='bar', x='feature', y='abs_importance')

Predict Test

In [ ]:
x_test = df_test.drop(id_features + categorical_features, axis=1)
x_test.fillna(0, inplace=True)

In [ ]:
pred_test = pd.DataFrame(model.predict(x_test), df_test.index)

In [ ]:
pred_test = pred_test.clip(lower=0, upper=1)

In [ ]:
pred_test

In [ ]:
df_sub['winPlacePerc'] = pred_test[0]

In [ ]:
df_sub.to_csv('lm_submission.csv', index=False)